In [3]:
import gensim
from tqdm import tqdm

### Wikipedia dump
- Download ```kawiki-latest-pages-articles-multistream.xml``` files from https://dumps.wikimedia.org/kawiki/latest/
- Parse text from xml files using WikiExtractor - https://github.com/attardi/wikiextractor
- Load ```kawiki.txt``` file

In [4]:
with open("../txt/kawiki.txt", "r", encoding="utf-8") as file:
    document = file.read()

document[:500]

' \nედუარდ შევარდნაძე\n\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი, 1928, მამათი, დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი, 2014, თბილისი) — ქართველი პოლიტიკოსი და სახელმწიფო მოღვაწე. 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი, 1995–2003 წლებში საქართველოს პრეზიდენტი.\n\nსკკპ წევრი 1948 წლიდან. 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად. IX-XI მოწვევების უმაღლესი საბჭოს დეპუტატი. სოციალისტური შრომის გმირი (1981). სკკპ ცკ-'

### Remove Stopwords
- Use ```stopwords.txt``` file to clean up Georgian stopwords from the text

In [5]:
with open("../txt/stopwords.txt", "r", encoding="utf-8") as file:
    stopwords = [line.replace('\n','') for line in file.readlines()]
stopwords[:5]

['ა.შ.', 'აგერ', 'აგრეთვე', 'ალბათ', 'ამაზე']

In [102]:
import re
for s in tqdm(stopwords):
    document = re.sub(r"\b" + s + r"\b", '', document.replace('\n', ' '))

document[:500]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 373/373 [12:32<00:00,  2.02s/it]


'  ედუარდ შევარდნაძე  ედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი, 1928, მამათი, დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი, 2014, თბილისი) — ქართველი პოლიტიკოსი  სახელმწიფო მოღვაწე. 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი, 1995–2003 წლებში საქართველოს პრეზიდენტი.  სკკპ წევრი 1948 წლიდან. 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად. IX-XI მოწვევების უმაღლესი საბჭოს დეპუტატი. სოციალისტური შრომის გმირი (1981). სკკპ ცკ-ის'

### Divide the document into sentences

In [114]:
sentences = re.split(r"(?<=\w\w\w)\.", re.sub(r'[^ა-ჰ0-9\.\-—–\s]',' ', document))
sentences[:5]

['  ედუარდ შევარდნაძე  ედუარდ ამბროსის ძე შევარდნაძე  დ. 25 იანვარი  1928  მამათი  დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი  2014  თბილისი  — ქართველი პოლიტიკოსი  სახელმწიფო მოღვაწე',
 ' 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი  1995–2003 წლებში საქართველოს პრეზიდენტი',
 '  სკკპ წევრი 1948 წლიდან',
 ' 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად',
 '   -   მოწვევების უმაღლესი საბჭოს დეპუტატი']

In [115]:
len(sentences)

1057879

In [119]:
def process_lines(sentences):
    for line in tqdm(sentences):
        yield gensim.utils.simple_preprocess(line, max_len=100)

In [120]:
tokens = list(process_lines(sentences))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1057879/1057879 [00:16<00:00, 64324.39it/s]


### Training
Common parameters to tune:
- min_count (int) – Ignores all words with total frequency lower than this.
- window (int) – The maximum distance between the current and predicted word within a sentence.
- size (int) – Dimensionality of the feature vectors.
- compute_loss (bool) – If True, computes and stores loss value which can be retrieved using model.get_latest_training_loss().
- workers (int) – Use these many worker threads to train the model (=faster training with multicore machines).

Defaults:
```class gensim.models.word2vec.Word2Vec(sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=())```

In [135]:
def accuracy(model):
    accuracy = model.wv.accuracy('../txt/questions-geography-ka.txt')
    sum_corr = len(accuracy[-1]['correct'])
    sum_incorr = len(accuracy[-1]['incorrect'])
    total = sum_corr + sum_incorr
    percent = lambda a: a / total * 100 if total != 0 else 0
    print('- TS: {}, Correct: {:.4f}%, Incorrect: {:.4f}%'.format(total, percent(sum_corr), percent(sum_incorr)))
    return percent(sum_corr)

In [123]:
from contextlib import contextmanager
import time
@contextmanager
def timeit(s):
    start = time.time()
    yield
    print(f"[{time.time()-start:3.0f}s] {s}\n\n")

In [157]:
with timeit(f" Done."):
    model = gensim.models.Word2Vec(tokens, size=300, min_count=2, window=20, workers=12)
    model.train(tokens, total_examples=len(tokens), epochs=10)
    accuracy(model)

- TS: 1265, Correct: 37.5494%, Incorrect: 62.4506%
[198s]  Done.




### Hyperparameter tuning

In [251]:
# Store results in global variable
results = []

In [ ]:
d = 200
mc = 100
w = 20

dims = [50,100,200,300]
mincounts = [2, 8, 16, 64]
windows = [2, 4, 8, 16, 32]

pbar = tqdm(total=len(dims)*len(mincounts)*len(windows))

for i,d in enumerate(dims):
    for j,mc in enumerate(mincounts):
        for k,w in enumerate(windows):
            pbar.update(1)
            with timeit(f"[S:{d} MC:{mc} W:{w}] Model training complete."):
                model = gensim.models.Word2Vec(tokens, size=d, min_count=mc, window=w, workers=12, compute_loss=True)
                model.train(tokens, total_examples=len(tokens), epochs=10)
                acc = accuracy(model)
                print(f"- Vocab size:{len(model.wv.vocab)}\n- Compute Loss: {model.get_latest_training_loss()}")
                results.append({
                    'size':d,
                    'min_count':mc,
                    'window':w,
                    'vocab':len(model.wv.vocab),
                    'accuracy':acc,
                    'loss':model.get_latest_training_loss()
                })
                
pbar.close()




  0%|                                                                                                                                                                                | 0/80 [00:00<?, ?it/s]

- TS: 1265, Correct: 18.1028%, Incorrect: 81.8972%
- Vocab size:356756
- Compute Loss: 0.0
[ 98s] [S:50 MC:2 W:2] Model training complete.







  2%|████▏                                                                                                                                                                 | 2/80 [01:37<1:03:25, 48.79s/it]

- TS: 1265, Correct: 24.0316%, Incorrect: 75.9684%
- Vocab size:356756
- Compute Loss: 0.0
[ 96s] [S:50 MC:2 W:4] Model training complete.







  4%|██████▏                                                                                                                                                               | 3/80 [03:13<1:20:36, 62.81s/it]

...




 24%|███████████████████████████████████████▏                                                                                                                             | 19/80 [23:55<1:03:38, 62.61s/it]

- TS: 657, Correct: 32.8767%, Incorrect: 67.1233%
- Vocab size:22082
- Compute Loss: 0.0
[ 60s] [S:50 MC:64 W:16] Model training complete.







 25%|█████████████████████████████████████████▎                                                                                                                           | 20/80 [24:54<1:01:46, 61.78s/it]

- TS: 657, Correct: 30.5936%, Incorrect: 69.4064%
- Vocab size:22082
- Compute Loss: 0.0
[ 58s] [S:50 MC:64 W:32] Model training complete.







 26%|███████████████████████████████████████████▊                                                                                                                           | 21/80 [25:52<59:29, 60.51s/it]

In [ ]:
df = pd.DataFrame(results)
df.to_csv("results.csv")

### Model testing

In [228]:
w1 = ['საფერავი']
model.wv.most_similar(positive=w1, topn=5)

[('რქაწითელი', 0.8808282613754272),
 ('ციცქა', 0.867584228515625),
 ('კრახუნა', 0.860209584236145),
 ('ცოლიკოური', 0.8515080213546753),
 ('ქინძმარაული', 0.8172898888587952)]

In [237]:
w1 = ['გიორგი', 'აღმაშენებელი']
w2 = ['დავით']
model.wv.most_similar(positive=w1, negative=w2, topn=10)

[('მერჩულეს', 0.5412800312042236),
 ('დანელიას', 0.509667694568634),
 ('მერჩულიულნის', 0.4998641908168793),
 ('ბრწყინვალის', 0.4979133605957031),
 ('მთაწმიდელის', 0.48206937313079834),
 ('ასანიშვილი', 0.4561194181442261),
 ('სააკაძე', 0.45581313967704773),
 ('მერჩულე', 0.4496610760688782),
 ('მთაწმიდლის', 0.446776807308197),
 ('პერანგითა', 0.4463770389556885)]

In [207]:
def analogie(a, b, c, model, **kwargs):
    return model.wv.most_similar(positive=[b,c], negative=[a], **kwargs)

In [239]:
analogie("თბილისი", "საქართველო", "კიევი", model, topn=2)

[('უკრაინა', 0.5557381510734558), ('პოლტავა', 0.5374630093574524)]

In [249]:
analogie("ცა", "ცისფერი", "ზღვა", model, topn=2)

[('ლურჯი', 0.45027443766593933), ('სალაკა', 0.44323498010635376)]

### Persist model
- and export as .tsv for http://projector.tensorflow.org/

In [145]:
with timeit("Model saved"):
    model.save("../model/kawiki_1250MB")

[ 11s] Model saved




In [129]:
def word2vec2tensor(model, tensor_filename, binary=False):
    outfiletsv = tensor_filename + '_tensor.tsv'
    outfiletsvmeta = tensor_filename + '_metadata.tsv'

    with open(outfiletsv, 'w+', encoding='utf-8') as file_vector:
        with open(outfiletsvmeta, 'w+', encoding='utf-8') as file_metadata:
            file_metadata.write('word\tcount\n')
            for word in tqdm(model.wv.index2word):
                wordstring = gensim.utils.to_utf8(word).decode("utf-8")
                countstring = str(model.wv.vocab[word].count)
                file_metadata.write(wordstring + '\t' + countstring + '\n')
                vector_row = '\t'.join(str(x) for x in model.wv[word])
                file_vector.write(vector_row + '\n')

In [144]:
word2vec2tensor(model,"../tsv/kawiki_1250MB")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356756/356756 [00:53<00:00, 6651.86it/s]
